<a href="https://colab.research.google.com/github/M4datta/Airline-Industry/blob/main/SalaryEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
    !tar xf spark-3.1.1-bin-hadoop3.2.tgz
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
    import findspark
    findspark.init()
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.master("local[*]").getOrCreate()
    spark

In [4]:
mydata=spark.read.format("csv").option("header","true").load("original.csv")

In [15]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [119]:
from pyspark.sql.functions import *
mydata2 = mydata.withColumn("clean_city", when(mydata.City.isNull(), 'Unknown').otherwise(mydata.City))

In [120]:
mydata2=mydata2.filter(mydata2.JobTitle.isNotNull())

In [10]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [131]:
mydata2= mydata2.withColumn('clean_salary',mydata2.Salary.substr(2,100).cast('float'))
df=mydata2
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+-----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|              lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+-----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|       50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|       48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|       44.5047212|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      n

In [134]:
mean= mydata2.groupBy().avg('clean_salary').take(1)[0][0]

In [133]:
print(mean)

<function mean at 0x7a9db146beb0>


In [135]:
from pyspark.sql.functions import lit
mydata2=mydata2.withColumn('new_salary', when(mydata2.clean_salary.isNull(),lit(mean)).otherwise(mydata2.clean_salary))

In [16]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 3

In [17]:
import numpy as np
latitudes=mydata2.select('latitude')

In [18]:
latitudes.show()

+----------+
|  latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|      null|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
+----------+
only showing top 20 rows



In [19]:
latitudes=latitudes.filter(latitudes.latitude.isNotNull())

In [20]:
latitudes.show()

+----------+
|  latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
|48.4902808|
+----------+
only showing top 20 rows



In [22]:
latitudes=latitudes.withColumn('latitude2',latitudes.latitude.cast('float')).select('latitude2')

In [23]:
latitudes.show()

+----------+
| latitude2|
+----------+
| 50.577408|
|  48.82316|
| 44.504723|
| 53.426613|
| 45.190517|
| 32.027935|
|  4.272793|
|     -5.85|
|  39.17238|
|  49.81518|
|  42.10148|
|  49.79233|
| 43.494576|
| 52.744167|
| 38.696247|
|-7.7232566|
| 40.717205|
|  49.16291|
| 40.757683|
|  48.49028|
+----------+
only showing top 20 rows



In [24]:
median=np.median(latitudes.collect())

In [25]:
print(median)

31.93397331237793


In [128]:
mydata2= mydata2.withColumn('lat', when(mydata2.Latitude.isNull(), lit(median)).otherwise(mydata2.Latitude))

In [127]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [136]:
import pyspark.sql.functions as sqlfunc
genders=mydata2.groupBy('gender').agg(sqlfunc.avg('new_salary').alias('AvgSalary'))

In [49]:
genders.show()

+------+------------------+
|gender|         AvgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [60]:
df= mydata2.withColumn('female_salary',when(mydata2.gender=='Female',mydata2.new_salary).otherwise(lit(0)))

In [61]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|              lat|   female_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|       50.5774075|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|       48.8231572|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...

In [138]:
df1=df.groupBy('gender').agg(sqlfunc.sum('clean_salary').alias('total'),
                             sqlfunc.avg('clean_salary').alias('average'))
df1.show()

+------+--------------------+------------------+
|gender|               total|           average|
+------+--------------------+------------------+
|Female|2.7281852561523438E7|55677.250125558036|
|  Male|2.8123435678710938E7| 55361.09385573019|
+------+--------------------+------------------+



In [62]:
df=df.withColumn('male_salary',when(df.gender=='Male',df.new_salary).otherwise(lit(0)))

In [53]:
df=df.groupBy('JobTitle').agg(sqlfunc.avg('female_salary').alias('final_female_salary'),sqlfunc.avg('male_salary').alias('final_male_salary'))

In [116]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [64]:
cityavg=mydata2.groupBy('City').agg(sqlfunc.avg('new_Salary').alias('avgsalary'))

In [65]:
cityavg=cityavg.sort(col('avgsalary').desc())

In [66]:
cityavg.show()

+-----------------+-------------+
|             City|    avgsalary|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows



In [67]:
df=spark.read.format("csv").option("header","true").load("original.csv")
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [68]:
df2= spark.read.csv("original.csv",header=True)
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [71]:
df2.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [76]:
from pyspark.sql.types import *
myschema= StructType([
    StructField('id', IntegerType()),
    StructField('first_name', StringType()),
    StructField('last_name',StringType()),
    StructField('gender',StringType()),
    StructField('city',StringType()),
    StructField('job_title',StringType()),
    StructField('Salary',StringType()),
    StructField('latitude',StringType()),
    StructField('longitude',FloatType())
])

In [77]:
df3=spark.read.csv("original.csv", header=True, schema=myschema)
df3.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   Salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [78]:
df3.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('city', 'string'),
 ('job_title', 'string'),
 ('Salary', 'string'),
 ('latitude', 'string'),
 ('longitude', 'float')]

In [80]:
df.head(6)

[Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184'),
 Row(id='2', first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude='48.8231572', Longitude='103.5218199'),
 Row(id='3', first_name='Alvera', last_name='Di Boldi', gender='Female', City=None, JobTitle=None, Salary='$57576.52', Latitude='39.9947462', Longitude='116.3397725'),
 Row(id='4', first_name='Shannon', last_name="O'Griffin", gender='Male', City='Divnomorskoye', JobTitle='Budget/Accounting Analyst II', Salary='$61489.23', Latitude='44.5047212', Longitude='38.1300171'),
 Row(id='5', first_name='Sherwood', last_name='Macieja', gender='Male', City='Mytishchi', JobTitle='VP Sales', Salary='$63863.09', Latitude=None, Longitude='37.6489954'),
 Row(id='6', first_name='Maris', last_name='Folk', gender='Female', City='Kin

In [81]:
df.first()

Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184')

In [82]:
df.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|         Latitude|         Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|              999|              1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null|25.43151724234234|43.337564614499996|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null| 24.5790825486909| 69.42064539970089|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|       -0.6256517|        -0.4889547|

In [83]:
df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [84]:
df_dropped=df.na.drop()
df_dropped.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36|45.1905186|  0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Manage...|$73697.10| 32.027934| 106.657113|
| 10|      Bran|   Trahear|  Male|       Arbeláez|Mecha

In [85]:
df_no_duplicates=df.dropDuplicates()
df_no_duplicates.show()

+---+----------+----------+------+----------------+--------------------+---------+----------+------------+
| id|first_name| last_name|gender|            City|            JobTitle|   Salary|  Latitude|   Longitude|
+---+----------+----------+------+----------------+--------------------+---------+----------+------------+
|167|    Buiron| Franzotto|  Male|         Prusice|    Health Coach III|$55731.78|51.3704296|  16.9606267|
|610|    Waylen| O' Clovan|  Male|           Khyzy|   Marketing Manager|$73169.48|40.9109489|  49.0729264|
|239|  Vivyanne|  Astridge|Female|         Baoshui|        Food Chemist|$11084.34| 30.513598|  110.361715|
|256|    Stearn|    Niesel|  Male|         Lungmar| Software Consultant|$31406.56|  28.85816|    89.94036|
|575|    Richie|  Hellicar|  Male|         Xichuan|        Developer IV|$40279.59|  33.13782|  111.490964|
|578|    Mignon|    Debnam|Female|   Guajará Mirim|       Social Worker|$81607.74|-10.789321| -65.3301049|
|784|  Marjorie|  Dekeyser|Female|   

In [87]:
df_renamed=df.withColumnRenamed('first_name','fn')
df_renamed.show()

+---+--------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|      fn| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+--------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1| Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|  Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4| Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|   Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|   Masha|    Divers|Female|         Dachun|                null|$250

In [89]:
df_filter=df.filter((df.first_name=='Alvera'))
df_filter.show()

+---+----------+---------+------+----+--------+---------+----------+-----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----+--------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.9947462|116.3397725|
+---+----------+---------+------+----+--------+---------+----------+-----------+



In [90]:
df_filter=df.filter((df.first_name.like("%lvera")))
df_filter.show()

+---+----------+---------+------+----+--------+---------+----------+-----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----+--------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.9947462|116.3397725|
+---+----------+---------+------+----+--------+---------+----------+-----------+



In [91]:
df_filter=df.filter((df.first_name.endswith('din')))
df_filter.show()

+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
| id|first_name|    last_name|gender|       City|JobTitle|   Salary|   Latitude| Longitude|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
|901|     Aldin|Matuszkiewicz|  Male|East London|Operator|$41468.83|-32.9549324|27.9319131|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+



In [93]:
df_filter=df.filter((df.first_name.startswith('Alv')))
df_filter.show()

+---+----------+---------+------+----------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|      City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----------+--------------------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|      null|                null|$57576.52|39.9947462|116.3397725|
| 81|     Alvin|    Doman|  Male|      Niny|Research Assistant I|$53258.86|44.4868448| 43.9408057|
|775|   Alverta| MacNulty|Female|Megalópoli| Geological Engineer|$17299.62| 37.401244| 22.1364871|
+---+----------+---------+------+----------+--------------------+---------+----------+-----------+



In [94]:
df_filter=df.filter((df.id.between(1,5)))
df_filter.show()

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+



In [98]:
df_substr=df.select(df.first_name,df.first_name.substr(1,5).alias('name'))
df_substr.show()

+----------+-----+
|first_name| name|
+----------+-----+
|   Melinde|Melin|
|  Kimberly|Kimbe|
|    Alvera|Alver|
|   Shannon|Shann|
|  Sherwood|Sherw|
|     Maris|Maris|
|     Masha|Masha|
|   Goddart|Godda|
|      Roth| Roth|
|      Bran| Bran|
|    Kylynn|Kylyn|
|       Rey|  Rey|
|      Kerr| Kerr|
|    Mickie|Micki|
|    Kaspar|Kaspa|
|    Norbie|Norbi|
|    Claude|Claud|
|     Thain|Thain|
|  Tiffanie|Tiffa|
|    Ettore|Ettor|
+----------+-----+
only showing top 20 rows



In [106]:
df_filter=df.filter((df.first_name.isin('Aldin', 'Valma'))| (df.City.like('%ondon')))
df_filter.show()

+---+----------+-------------+------+-----------+----------------+---------+-----------+-----------+
| id|first_name|    last_name|gender|       City|        JobTitle|   Salary|   Latitude|  Longitude|
+---+----------+-------------+------+-----------+----------------+---------+-----------+-----------+
|569|     Valma|      Bratton|Female|  Kurayoshi|Web Developer II|$32665.89| 35.4499037|133.7613311|
|901|     Aldin|Matuszkiewicz|  Male|East London|        Operator|$41468.83|-32.9549324| 27.9319131|
+---+----------+-------------+------+-----------+----------------+---------+-----------+-----------+



In [107]:
df_filter=df.filter((df.id>10)&(df.id<100))
df_filter.show()

+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|          City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+
| 11|    Kylynn|  Lockart|Female|      El Cardo|Nuclear Power Eng...|$13604.63|     -5.85|-79.8833329|
| 12|       Rey|   Meharg|Female|   Wangqingtuo|Systems Administr...|$73423.70| 39.172378| 116.931607|
| 13|      Kerr|   Braden|  Male|     Sułkowice|Compensation Analyst|$33432.99|49.8151822| 19.3771749|
| 14|    Mickie|Whanstall|  Male|   Springfield|Assistant Media P...|$50838.53|42.1014803|-72.5766759|
| 15|    Kaspar|    Pally|  Male|        Chrást|  Analyst Programmer|$40163.03|49.7923299| 13.4915324|
| 16|    Norbie|   Gwyllt|  Male|        Xijiao|              Editor|$32492.73|43.4945737|  5.8978018|
| 17|    Claude|   Briant|Female|     Mieścisko|Research Assistan...|$518

In [108]:
df.registerTempTable("original")

In [110]:
query1=spark.sql('select* from original')
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [113]:
query2=spark.sql('select concat(first_name,"",last_name) as full_name from original where gender="Female"')

In [114]:
query2.show()

+------------------+
|         full_name|
+------------------+
|  MelindeShilburne|
|KimberlyVon Welden|
|    AlveraDi Boldi|
|         MarisFolk|
|       MashaDivers|
|     KylynnLockart|
|         ReyMeharg|
|      ClaudeBriant|
|  TiffaniePattison|
|    LurleenJanczak|
|      NicholHoltum|
|       ShaunBridle|
|     LeandraAnfrey|
|    JaquelynHazard|
|  PrudenceHonacker|
|       ChereyLiger|
|          NedaKrop|
|    BarbiFattorini|
|   LonnieTownshend|
|    ValidaSalzberg|
+------------------+
only showing top 20 rows



In [123]:
df=df.withColumn('monthly_salary', df.clean_salary/12)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|    monthly_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18| 4786.514973958333|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|    5237.216796875|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|   5124.1025390625|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09| 

In [125]:
df=df.withColumn('are_they_female',when(df.gender=='Female','Yes').otherwise('no'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+------------------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|    monthly_salary|are_they_female|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+------------------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18| 4786.514973958333|            Yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|    5237.216796875|            Yes|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|   5124.1025390625|   

In [139]:
df1.write.csv('df1.csv')
df1.write.json('df1.json')
df1.write.parquet('df1.parquet')